In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import os
import re
import matplotlib.pyplot as plt
import networkx as nx
from sklearn.preprocessing import MinMaxScaler
from itertools import combinations
from sklearn.metrics.pairwise import pairwise_distances
import scipy.stats as stats
from minisom import MiniSom
import ot
import cvxpy as cp

In [2]:
from joblib import Parallel, delayed
import itertools
from tqdm import tqdm
from itertools import product
from tqdm_joblib import tqdm_joblib
from scipy.stats import binom, kstest, wasserstein_distance

/opt/anaconda3/envs/myenv2/lib/python3.13/site-packages/tqdm_joblib/__init__.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [6]:
def construct_P(N, s):
    return np.delete(np.eye(N), s, axis=0)

def evaluate_graph_stability(A, X, P, lambda_reg):
    N_new = P.shape[0]  
    X_tilde = P @ X
    A_proj = P @ A @ P.T

    A_tilde = cp.Variable((N_new, N_new), symmetric=True)
    D_tilde = cp.diag(cp.sum(A_tilde, axis=1))
    L_tilde = cp.Variable((N_new, N_new), symmetric=True)

    objective = cp.trace(X_tilde.T @ L_tilde @ X_tilde) + lambda_reg * cp.norm(A_tilde - A_proj, 'fro')

    constraints = [
        A_tilde >= 0,  
        cp.diag(A_tilde) == 0,  
        L_tilde @ np.ones((N_new, 1)) == 0,  
        L_tilde == D_tilde - A_tilde,
        L_tilde >> 0  
    ]
    #  L_ij <= 0 para i != j
    for i in range(N_new):
        for j in range(N_new):
            if i != j:
                constraints.append(L_tilde[i, j] <= 0)

    
    problem = cp.Problem(cp.Minimize(objective), constraints)
    problem.solve(solver=cp.SCS, eps=1e-3)


    
    A_tilde_value = np.maximum(A_tilde.value, 0)
    A_tilde_value_np =np.array(A_tilde_value)

    return A_tilde_value_np

 Instalar: pip install ipywidgets

 
N,F:(20, 50), (40,80),(60, 100), (80,120) y (100, 150)  
SNR:  inf, 20, 0 y -5  
Node dropout: 0.2, 0.4, 0.6, 0.7 y 0.8  
50 times 

In [9]:
import os
import gc
import time
import tracemalloc
from itertools import product

import numpy as np
import networkx as nx
import pandas as pd
import cvxpy as cp

from scipy.stats import binom, kstest, wasserstein_distance

from joblib import Parallel, delayed
from tqdm import tqdm
from tqdm_joblib import tqdm_joblib

In [10]:
class GraphData:
"""
Stores:
  - A: adjacency matrix of the graph (numpy array)
  - p: probability used
  - L: Laplacian
  - eigvals, eigvecs: decomposition of L (saved for reuse)
"""

    def __init__(self, N, factor=2):
        
        self.N = N
        self.p = self.get_p(N, factor)      
        self.A = self.generate_erdos_renyi_graph(N, self.p)
        self.L = self.compute_laplacian(self.A)

        self.eigvals, self.eigvecs = np.linalg.eigh(self.L)

    @staticmethod
    def get_p(N, factor=2):
        """
        p = factor * (log N / N-1).
        """
        return factor * (np.log(N)/(N-1))

    @staticmethod
    def generate_erdos_renyi_graph(N, p):
        G = nx.erdos_renyi_graph(N, p)
        return nx.to_numpy_array(G)

    @staticmethod
    def compute_laplacian(A):
        D = np.diag(np.sum(A, axis=1))
        return D - A

    def generate_X_with_SNR(self, F, snr_db, return_noise_stats=False):
        """
        1. L = V Λ V^T.
        2. h ~ N(0, Λ^+), sea X_clean = V h.
        3. scale X_clean so that it has variance ≈1 per node.
        4. add Gaussian noise with variance = var(X_clean)/snr_linear.
        5. return X_noisy = X_clean + noise.
        """
        #N = L.shape[0]
        #eigvals, eigvecs = np.linalg.eigh(L)
        N = self.N
        eigvals = self.eigvals
        eigvecs = self.eigvecs
        
        lambda_inv = np.zeros_like(eigvals)
        for i, val in enumerate(eigvals):
            if val > 1e-6:
                lambda_inv[i] = 1.0 / val
            else:
                lambda_inv[i] = 0.0
        Lambda_inv = np.diag(lambda_inv)
        
        # h ~ N(0, Lambda^+)
        h = np.random.multivariate_normal(mean=np.zeros(N), cov=Lambda_inv, size=F).T  # shape (N, F) F vectores h de dimensión N
        # clean signal
        X_clean = eigvecs @ h  # (N, F)

        X_zero_mean = X_clean - X_clean.mean(axis=1, keepdims=True)
        std_row = X_zero_mean.std(axis=1, ddof=1, keepdims=True) + 1e-12
        X_norm = X_zero_mean / std_row

        snr_linear = 10 ** (snr_db / 10.0)
        power_signal = np.mean(X_norm**2)  # ≈1

        sigma_noise = np.sqrt(power_signal / snr_linear)
        noise = np.random.normal(loc=0.0, scale=sigma_noise, size=X_norm.shape)
        noise_power = np.mean(noise**2)
        snr_real = power_signal / noise_power if noise_power > 1e-12 else np.inf

        X_noisy = X_norm + noise

        if return_noise_stats:
            stats = {
                'snr_db': snr_db,
                'sigma_noise': sigma_noise,
                'signal_power': power_signal,
                'noise_power': noise_power,
                'snr_linear_real': snr_real,
                # etc.
            }
            return X_noisy, stats
        else:
            return X_noisy

In [11]:
def construct_P(N, s):
    return np.delete(np.eye(N), s, axis=0)
    
def remap_A_hat(Atil_small, removed):
    """
    Given the reconstructed matrix Atil_small of size (N-1)x(N-1),
    rebuilds the N×N matrix by inserting zero rows/columns at the 'removed' index.
    """
    N_minus_1 = Atil_small.shape[0]
    N = N_minus_1 + 1
    A_hat = np.zeros((N, N))
    idx = list(range(N))
    idx.pop(removed)

    for i_small, i_big in enumerate(idx):
        for j_small, j_big in enumerate(idx):
            A_hat[i_big, j_big] = Atil_small[i_small, j_small]

    return A_hat


def binarize(A, eps=1e-8):
    """
    Binarizes A: returns a matrix of the same size with 1 where |A[i,j]| > eps, and 0 otherwise.
    """
    return (np.abs(A) > eps).astype(float)


def mask_node(A, removed):
    """
    Given A (N×N), sets to zero the row and column at 'removed' (simulates the removal of that node).
    """
    M = A.copy()
    M[removed, :] = 0
    M[:, removed] = 0
    return M

def calculate_nmse_with_mask_and_binarize(Atil_small, A_full, removed, eps=1e-8):
    """
    1) Remaps Atil_small to the original size N×N,
    2) Binarizes A_hat and the masked A_full,
    3) Computes binarized NMSE: ||A_hat_bin - A_full_bin||_F / ||A_full_bin||_F

    Returns (nmse, A_hat_bin, A_full_bin).
    """
    A_hat = remap_A_hat(Atil_small, removed)
    A_hat_bin = binarize(A_hat, eps)

    A_full_masked = mask_node(A_full, removed)
    A_full_bin = binarize(A_full_masked, eps)
    
    diff = np.linalg.norm(A_hat_bin - A_full_bin, 'fro')
    normA = np.linalg.norm(A_full_bin, 'fro')
    return diff / normA, A_hat_bin, A_full_bin

In [11]:
# not required 
def choose_best_lambda_median_sampled(A, X, lambda_list, threshold=0.2, eps=1e-6,sample_fraction=0.5, random_seed=0):
  """
  For each lambda in lambda_list:
    - Sample a fraction `sample_fraction` of nodes at random (without replacement).
    - For each node in that sample:
      * Build P, run inference, and compute binarized NMSE.
    - Take the median of all obtained NMSE values.

  Finally, return:
    best_lambda = the λ whose MedNMSE is the minimum among those below `threshold` (if any),
                  otherwise the one with the lowest median NMSE.
    medians = dictionary { lambda: median_NMSE }.
  """

    N = A.shape[0]
    rng = np.random.default_rng(random_seed)
    n = max(1, int(N * sample_fraction))
    nodes = rng.choice(N, size=n, replace=False)

    medians = {}
    for lam in lambda_list:
        nmse_vals = []
        for node in nodes:
            P = construct_P(N, node)
            A_tilde = evaluate_graph_stability(A, X, P, lam)
            nmse, _, _ = calculate_nmse_with_mask_and_binarize(A_tilde, A, node, eps=eps)
            nmse_vals.append(nmse)
        medians[lam] = np.median(nmse_vals)
    valid = [l for l, m in medians.items() if m <= threshold]
    best = min(valid, key=lambda l: medians[l]) if valid else min(medians, key=medians.get)
    
    return best, medians

In [12]:
# Fast version
def evaluate_graph_stability_fast(A, X, P, lambda_reg, eps=1e-3):

    X_tilde = P @ X                      # (N-1)×F
    A_proj   = P @ A @ P.T               # (N-1)×(N-1)
    N_new    = X_tilde.shape[0]

    diff    = X_tilde[:, None, :] - X_tilde[None, :, :]  # (N-1,N-1,F)
    W_np    = np.sum(diff**2, axis=2)                    # Z_{ij} = ||x_i - x_j||^2

    A_tilde = cp.Variable((N_new, N_new), symmetric=True)

    obj = 0.5 * cp.sum(cp.multiply(W_np, A_tilde)) \
          + lambda_reg * cp.norm(A_tilde - A_proj, 'fro')

    constraints = [
        A_tilde >= 0,            
        cp.diag(A_tilde) == 0    
    ]

    prob = cp.Problem(cp.Minimize(obj), constraints)
    prob.solve(solver=cp.SCS, eps=eps)
    A_sol = np.maximum(A_tilde.value, 0.0)
    return A_sol


In [18]:
import time
import tracemalloc
from itertools import product
from joblib import Parallel, delayed
from tqdm.auto import tqdm
from sklearn.metrics import roc_auc_score
from scipy.stats import wasserstein_distance

# single_experiment_reorganized_fast_parallel

def single_experiment_reorganized_fast_parallel(
    N, F, sigma_db, lambda_list,
    threshold=0.55, sample_fraction=0.8,
    eps=1e-6, seed=0, n_jobs=1
):
    tracemalloc.start()
    t0 = time.perf_counter()

    graph_data = GraphData(N=N, factor=2)
    A          = graph_data.A
    p_used     = graph_data.p
    try:
        X, noise_stats = graph_data.generate_X_with_SNR(
            F=F, snr_db=sigma_db, return_noise_stats=True
        )
    except TypeError:
        X = graph_data.generate_X_with_SNR(F=F, snr_db=sigma_db)
        noise_stats = {}

    G_gt           = nx.from_numpy_array(binarize(A))
    clustering_gt  = nx.average_clustering(G_gt)
    betweenness_gt = np.mean(list(nx.betweenness_centrality(G_gt).values()))
    if nx.is_connected(G_gt):
        path_gt = nx.average_shortest_path_length(G_gt)
    else:
        cc = max(nx.connected_components(G_gt), key=len)
        path_gt = nx.average_shortest_path_length(G_gt.subgraph(cc))

    rng    = np.random.default_rng(seed)
    nodes  = rng.choice(N, size=int(np.ceil(sample_fraction * N)), replace=False)
    P_list = [construct_P(N, i) for i in range(N)]

    Aorig_b_list = []
    deg_gt_list  = []
    iu, ju = np.triu_indices(N, k=1)
    for i in range(N):
        _, _, Aorig_b = calculate_nmse_with_mask_and_binarize(
            np.zeros((N-1, N-1)), A, i, eps=eps
        )
        Aorig_b_list.append(Aorig_b)
        deg_gt_list.append(Aorig_b.sum(axis=1))
    y_true_full = Aorig_b_list[0][iu, ju]

    t1 = time.perf_counter()

    #  Loop over λ
    lambda_results = []
    for lam in lambda_list:
        # Worker function for parallel execution
        def _worker(node):
            P = P_list[node]
            A_tilde = evaluate_graph_stability_fast(A, X, P, lam)

            nmse_val, Ahat_b, _ = calculate_nmse_with_mask_and_binarize(
                A_tilde, A, node, eps=eps
            )

            Ahat_cont = remap_A_hat(A_tilde, node)

            y_score = Ahat_cont[iu, ju]
            try:
                auc = roc_auc_score(y_true_full, y_score)
            except ValueError:
                auc = np.nan

            # Clustering, path and betweenness
            G_inf = nx.from_numpy_array(Ahat_b)
            C_inf = nx.average_clustering(G_inf)
            C_orig= nx.average_clustering(nx.from_numpy_array(Aorig_b_list[node]))
            def avg_path(G):
                if nx.is_connected(G):
                    return nx.average_shortest_path_length(G)
                cc = max(nx.connected_components(G), key=len)
                return nx.average_shortest_path_length(G.subgraph(cc))
            path_inf = avg_path(G_inf)
            path_orig= avg_path(nx.from_numpy_array(Aorig_b_list[node]))
            bc_inf  = np.mean(list(nx.betweenness_centrality(G_inf).values()))
            bc_orig = np.mean(list(nx.betweenness_centrality(nx.from_numpy_array(Aorig_b_list[node])).values()))

            return {
                'nmse':           nmse_val,
                'auc':            auc,
                'deg_gt_mean':    float(deg_gt.mean()),
                'deg_inf_mean':   float(deg_inf.mean()),
                'deg_gt_std':     float(deg_gt.std()),
                'deg_inf_std':    float(deg_inf.std()),
                'clustering_inf': C_inf,
                'clustering_orig':C_orig,
                'path_inf':       path_inf,
                'path_orig':      path_orig,
                'betw_inf':       bc_inf,
                'betw_orig':      bc_orig
            }

        t_start = time.perf_counter()
        metrics = Parallel(n_jobs=n_jobs)(
            delayed(_worker)(node)
            for node in tqdm(nodes, desc=f"λ={lam:.2e}", unit="node")
        )
        t_end = time.perf_counter()

        dfm = pd.DataFrame(metrics)
        agg = dfm.agg(['mean','median','std']).to_dict()
        row = {'lambda': lam, 'time_solve_nodes': t_end - t_start}
        for met, stats in agg.items():
            for stat, val in stats.items():
                row[f'{met}_{stat}'] = float(val)
        lambda_results.append(row)

    t2 = time.perf_counter()

    dfres = pd.DataFrame(lambda_results)
    valid = dfres[dfres['nmse_median'] <= threshold]
    best = valid.loc[valid['nmse_median'].idxmin()] if not valid.empty else dfres.loc[dfres['nmse_median'].idxmin()]

    result = {
        'N': N, 'F': F, 'p': p_used, 'sigma_db': sigma_db,
        'best_lambda':        float(best['lambda']),
        'nmse_median_best':   float(best['nmse_median']),
        'time_total':         t2 - t0,
        'time_data_and_prep': t1 - t0,
        'time_compute_metrics': t2 - t1,
        'clustering_gt':      clustering_gt,
        'betweenness_gt':     betweenness_gt,
        'path_gt':            path_gt
    }
    for k, v in best.items():
        if k != 'lambda':
            result[f'{k}'] = float(v)
    result.update(noise_stats)
    tracemalloc.stop()
    return result


In [19]:
from itertools import product
from joblib import Parallel, delayed
from tqdm.auto import tqdm

def run_stability_experiments_safe_save_csv(
    NF_list, sigma_list, dropout_list, lambda_list,
    threshold=0.55, eps=1e-6, repeats=10, seed=0,
    n_jobs=4, results_filepath="Resultados_estabilidad.csv"
):
    """
    save CSV. 
    
    """

    settings = [
        (N, F, sigma_db, drop, rep)
        for (N, F), sigma_db, drop, rep
        in product(NF_list, sigma_list, dropout_list, range(repeats))
    ]
    rng = np.random.default_rng(seed)

    if not os.path.exists(results_filepath):
        sample = single_experiment_reorganized_fast_parallel(
            N=NF_list[0][0], F=NF_list[0][1],
            sigma_db=sigma_list[0],
            lambda_list=lambda_list,
            threshold=threshold,
            sample_fraction=dropout_list[0],
            eps=eps,
            seed=seed,
            n_jobs=n_jobs
        )

        sample.update({
            "dropout_fraction": dropout_list[0],
            "rep": 0,
            "N": NF_list[0][0],
            "F": NF_list[0][1],
            "sigma_db": sigma_list[0]
        })
        pd.DataFrame([sample]).to_csv(results_filepath, index=False)
        done_keys = set()
    else:
        df_exist = pd.read_csv(results_filepath)
        done_keys = set(zip(
            df_exist["N"], df_exist["F"], df_exist["sigma_db"],
            df_exist["dropout_fraction"], df_exist["rep"]
        ))

    #  worker
    def run_one(cfg):
        N, F, sigma_db, drop, rep = cfg
        key = (N, F, sigma_db, drop, rep)
        if key in done_keys:
            return None
        row = single_experiment_reorganized_fast_parallel(
            N=N, F=F, sigma_db=sigma_db,
            lambda_list=lambda_list,
            threshold=threshold,
            sample_fraction=drop,
            eps=eps,
            seed=seed + rep,
            n_jobs=n_jobs
        )
        row.update({
            "dropout_fraction": drop,
            "rep": rep,
            "N": N, "F": F, "sigma_db": sigma_db
        })
        # append to CSV
        pd.DataFrame([row]).to_csv(
            results_filepath, mode="a", header=False, index=False
        )
        return row

    results = Parallel(n_jobs=n_jobs)(
        delayed(run_one)(cfg)
        for cfg in tqdm(settings, desc="Experiments", unit="cfg")
    )
    
    rows = [r for r in results if r is not None]
    return pd.DataFrame(rows)

In [20]:
def main():
    # === GRID OF TESTS ===
    NF_list      = [(20, 50), (40, 80), (60, 100), (80, 120), (100, 150)]
    sigma_list   = [np.inf, 20, 0, -5]
    dropout_list = [0.2, 0.4, 0.6, 0.8]
    lambda_list  = [1e-2, 1e-1, 2e-1, 1, 2, 5]

    threshold   = 0.5
    eps         = 1e-6
    repeats     = 50
    seed        = 42
    n_jobs      = 2
    results_filepath   = "Results_septiembreshort.csv"

    df_new = run_stability_experiments_safe_save_csv(
        NF_list=NF_list,
        sigma_list=sigma_list,
        dropout_list=dropout_list,
        lambda_list=lambda_list,
        threshold=threshold,
        eps=eps,
        repeats=repeats,
        seed=seed,
        n_jobs=n_jobs,
        results_filepath=results_filepath
    )
    print("Done. Results saved to", results_filepath)
if __name__ == "__main__":
    main()


λ=1.00e-02:   0%|          | 0/4 [00:00<?, ?node/s]

λ=1.00e-01:   0%|          | 0/4 [00:00<?, ?node/s]

λ=2.00e-01:   0%|          | 0/4 [00:00<?, ?node/s]

λ=1.00e+00:   0%|          | 0/4 [00:00<?, ?node/s]

λ=2.00e+00:   0%|          | 0/4 [00:00<?, ?node/s]

λ=5.00e+00:   0%|          | 0/4 [00:00<?, ?node/s]

Experiments:   0%|          | 0/2000 [00:00<?, ?cfg/s]

λ=5.00e+00: 100%|██████████| 4/4 [00:00<00:00, 35.27node/s]
/opt/anaconda3/envs/myenv2/lib/python3.13/site-packages/cvxpy/problems/problem.py:1481: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
λ=1.00e-02: 100%|██████████| 8/8 [00:00<00:00, 15.48node/s]

λ=1.00e+00: 100%|██████████| 16/16 [00:02<00:00,  7.13node/s]

λ=5.00e+00: 100%|██████████| 24/24 [00:08<00:00,  2.98node/s]

λ=5.00e+00: 100%|██████████| 40/40 [00:27<00:00,  1.45node/s]


Se agregaron 2000 registros nuevos al archivo 'Resultados_estabilidad_parcial_septiembreshort.csv'.
